## 1. نصب کتابخانه‌های مورد نیاز / Install Required Libraries

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import h5py
from pathlib import Path

# تنظیم مسیر سیستم / Set system path
sys.path.append('./system')

print("کتابخانه‌ها با موفقیت بارگذاری شدند / Libraries loaded successfully")

## 2. تولید داده‌های MNIST / Generate MNIST Dataset

In [ ]:
# بررسی وجود داده‌های MNIST / Check if MNIST data exists
if not os.path.exists('./dataset/MNIST'):
    print("در حال تولید داده‌های MNIST... / Generating MNIST data...")
    os.chdir('./dataset')
    !python generate_MNIST.py noniid - balance
    os.chdir('..')
else:
    print("داده‌های MNIST موجود است / MNIST data already exists")

## 3. تنظیمات آزمایش / Experiment Configuration

In [ ]:
# پارامترهای مشترک / Common parameters
CONFIG = {
    'dataset': 'MNIST',
    'model': 'CNN',
    'batch_size': 10,
    'learning_rate': 0.01,
    'num_clients': 20,
    'join_ratio': 0.5,  # نیمی از کلاینت‌ها در هر راund / Half of clients per round
    'global_rounds': 100,
    'local_epochs': 5,
    'num_classes': 10,
    'device': 'cuda',  # یا 'cpu' / or 'cpu'
    'eval_gap': 1,
    'times': 3,  # تعداد تکرار آزمایش / Number of experiment runs
}

# پارامترهای خاص SR-FedAvg / SR-FedAvg specific parameters
SR_CONFIG = {
    'sr_beta': 0.9,  # ضریب momentum برای Stein-Rule
}

print("تنظیمات آزمایش:")
print(json.dumps(CONFIG, indent=2, ensure_ascii=False))
print("\nتنظیمات SR-FedAvg:")
print(json.dumps(SR_CONFIG, indent=2, ensure_ascii=False))

## 4. اجرای FedAvg / Run FedAvg

In [ ]:
print("=" * 60)
print("شروع آزمایش FedAvg / Starting FedAvg Experiment")
print("=" * 60)

os.chdir('./system')

# اجرای FedAvg / Run FedAvg
!python main.py \
    -data {CONFIG['dataset']} \
    -m {CONFIG['model']} \
    -algo FedAvg \
    -gr {CONFIG['global_rounds']} \
    -ls {CONFIG['local_epochs']} \
    -lr {CONFIG['learning_rate']} \
    -lbs {CONFIG['batch_size']} \
    -nc {CONFIG['num_clients']} \
    -jr {CONFIG['join_ratio']} \
    -ncl {CONFIG['num_classes']} \
    -dev {CONFIG['device']} \
    -eg {CONFIG['eval_gap']} \
    -t {CONFIG['times']} \
    -go comparison

os.chdir('..')
print("\nآزمایش FedAvg به پایان رسید / FedAvg experiment completed")

## 5. اجرای SR-FedAvg / Run SR-FedAvg

In [ ]:
print("=" * 60)
print("شروع آزمایش SR-FedAvg / Starting SR-FedAvg Experiment")
print("=" * 60)

os.chdir('./system')

# اجرای SR-FedAvg / Run SR-FedAvg
!python main.py \
    -data {CONFIG['dataset']} \
    -m {CONFIG['model']} \
    -algo SR-FedAvg \
    -gr {CONFIG['global_rounds']} \
    -ls {CONFIG['local_epochs']} \
    -lr {CONFIG['learning_rate']} \
    -lbs {CONFIG['batch_size']} \
    -nc {CONFIG['num_clients']} \
    -jr {CONFIG['join_ratio']} \
    -ncl {CONFIG['num_classes']} \
    -dev {CONFIG['device']} \
    -eg {CONFIG['eval_gap']} \
    -t {CONFIG['times']} \
    -srbeta {SR_CONFIG['sr_beta']} \
    -go comparison

os.chdir('..')
print("\nآزمایش SR-FedAvg به پایان رسید / SR-FedAvg experiment completed")

## 6. خواندن نتایج / Load Results

In [ ]:
def load_results(dataset, algorithm, goal, times):
    """خواندن نتایج از فایل‌های h5 / Load results from h5 files"""
    results = {'test_acc': [], 'test_auc': [], 'train_loss': []}
    
    for t in range(times):
        filename = f"results/{dataset}_{algorithm}_{goal}_{t}.h5"
        
        if os.path.exists(filename):
            with h5py.File(filename, 'r') as f:
                results['test_acc'].append(np.array(f['rs_test_acc']))
                results['test_auc'].append(np.array(f['rs_test_auc']))
                results['train_loss'].append(np.array(f['rs_train_loss']))
        else:
            print(f"هشدار: فایل {filename} یافت نشد / Warning: File {filename} not found")
    
    # محاسبه میانگین / Calculate average
    if len(results['test_acc']) > 0:
        results['test_acc_mean'] = np.mean(results['test_acc'], axis=0)
        results['test_acc_std'] = np.std(results['test_acc'], axis=0)
        results['train_loss_mean'] = np.mean(results['train_loss'], axis=0)
        results['train_loss_std'] = np.std(results['train_loss'], axis=0)
    
    return results

# بارگذاری نتایج / Load results
fedavg_results = load_results(CONFIG['dataset'], 'FedAvg', 'comparison', CONFIG['times'])
srfedavg_results = load_results(CONFIG['dataset'], 'SR-FedAvg', 'comparison', CONFIG['times'])

print("نتایج با موفقیت بارگذاری شدند / Results loaded successfully")
print(f"FedAvg - بهترین دقت: {np.max(fedavg_results['test_acc_mean']):.4f}")
print(f"SR-FedAvg - بهترین دقت: {np.max(srfedavg_results['test_acc_mean']):.4f}")

## 7. نمایش نتایج / Visualize Results

In [ ]:
# تنظیمات نمودار / Plot settings
plt.rcParams['figure.figsize'] = (16, 6)
plt.rcParams['font.size'] = 12

fig, axes = plt.subplots(1, 2)

# نمودار دقت تست / Test Accuracy Plot
ax1 = axes[0]
rounds = range(len(fedavg_results['test_acc_mean']))

ax1.plot(rounds, fedavg_results['test_acc_mean'], 'b-', linewidth=2, label='FedAvg')
ax1.fill_between(rounds, 
                  fedavg_results['test_acc_mean'] - fedavg_results['test_acc_std'],
                  fedavg_results['test_acc_mean'] + fedavg_results['test_acc_std'],
                  alpha=0.2, color='blue')

ax1.plot(rounds, srfedavg_results['test_acc_mean'], 'r-', linewidth=2, label='SR-FedAvg')
ax1.fill_between(rounds,
                  srfedavg_results['test_acc_mean'] - srfedavg_results['test_acc_std'],
                  srfedavg_results['test_acc_mean'] + srfedavg_results['test_acc_std'],
                  alpha=0.2, color='red')

ax1.set_xlabel('Global Rounds (دور آموزش)', fontsize=14)
ax1.set_ylabel('Test Accuracy (دقت تست)', fontsize=14)
ax1.set_title('Test Accuracy Comparison\nمقایسه دقت تست', fontsize=16, fontweight='bold')
ax1.legend(fontsize=12)
ax1.grid(True, alpha=0.3)

# نمودار خطای آموزش / Training Loss Plot
ax2 = axes[1]
ax2.plot(rounds, fedavg_results['train_loss_mean'], 'b-', linewidth=2, label='FedAvg')
ax2.fill_between(rounds,
                  fedavg_results['train_loss_mean'] - fedavg_results['train_loss_std'],
                  fedavg_results['train_loss_mean'] + fedavg_results['train_loss_std'],
                  alpha=0.2, color='blue')

ax2.plot(rounds, srfedavg_results['train_loss_mean'], 'r-', linewidth=2, label='SR-FedAvg')
ax2.fill_between(rounds,
                  srfedavg_results['train_loss_mean'] - srfedavg_results['train_loss_std'],
                  srfedavg_results['train_loss_mean'] + srfedavg_results['train_loss_std'],
                  alpha=0.2, color='red')

ax2.set_xlabel('Global Rounds (دور آموزش)', fontsize=14)
ax2.set_ylabel('Training Loss (خطای آموزش)', fontsize=14)
ax2.set_title('Training Loss Comparison\nمقایسه خطای آموزش', fontsize=16, fontweight='bold')
ax2.legend(fontsize=12)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('comparison_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("نمودارها ذخیره شدند: comparison_results.png")

## 8. جدول مقایسه آماری / Statistical Comparison Table

In [ ]:
# محاسبه آماره‌ها / Calculate statistics
comparison_data = {
    'Metric': [
        'Final Test Accuracy (دقت نهایی)',
        'Best Test Accuracy (بهترین دقت)',
        'Final Train Loss (خطای نهایی)',
        'Best Train Loss (کمترین خطا)',
        'Convergence Round (دور همگرایی)'
    ],
    'FedAvg': [
        f"{fedavg_results['test_acc_mean'][-1]:.4f} ± {fedavg_results['test_acc_std'][-1]:.4f}",
        f"{np.max(fedavg_results['test_acc_mean']):.4f}",
        f"{fedavg_results['train_loss_mean'][-1]:.4f} ± {fedavg_results['train_loss_std'][-1]:.4f}",
        f"{np.min(fedavg_results['train_loss_mean']):.4f}",
        f"{np.argmax(fedavg_results['test_acc_mean'])}"
    ],
    'SR-FedAvg': [
        f"{srfedavg_results['test_acc_mean'][-1]:.4f} ± {srfedavg_results['test_acc_std'][-1]:.4f}",
        f"{np.max(srfedavg_results['test_acc_mean']):.4f}",
        f"{srfedavg_results['train_loss_mean'][-1]:.4f} ± {srfedavg_results['train_loss_std'][-1]:.4f}",
        f"{np.min(srfedavg_results['train_loss_mean']):.4f}",
        f"{np.argmax(srfedavg_results['test_acc_mean'])}"
    ]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("جدول مقایسه نتایج / Results Comparison Table")
print("="*80)
print(df_comparison.to_string(index=False))
print("="*80)

# محاسبه بهبود / Calculate improvement
improvement = (
    (np.max(srfedavg_results['test_acc_mean']) - np.max(fedavg_results['test_acc_mean'])) 
    / np.max(fedavg_results['test_acc_mean']) * 100
)

print(f"\n📊 بهبود SR-FedAvg نسبت به FedAvg: {improvement:+.2f}%")
print(f"📊 SR-FedAvg Improvement over FedAvg: {improvement:+.2f}%")

# ذخیره جدول / Save table
df_comparison.to_csv('comparison_table.csv', index=False)
print("\n✓ جدول در فایل comparison_table.csv ذخیره شد")

## 9. تحلیل پایداری / Stability Analysis

In [ ]:
# محاسبه واریانس در طول آموزش / Calculate variance during training
plt.figure(figsize=(12, 5))

# نمودار واریانس دقت / Accuracy variance plot
plt.subplot(1, 2, 1)
plt.plot(rounds, fedavg_results['test_acc_std'], 'b-', linewidth=2, label='FedAvg', marker='o', markersize=3)
plt.plot(rounds, srfedavg_results['test_acc_std'], 'r-', linewidth=2, label='SR-FedAvg', marker='s', markersize=3)
plt.xlabel('Global Rounds', fontsize=12)
plt.ylabel('Standard Deviation of Accuracy', fontsize=12)
plt.title('Stability: Test Accuracy Variance\nپایداری: واریانس دقت', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

# نمودار واریانس خطا / Loss variance plot
plt.subplot(1, 2, 2)
plt.plot(rounds, fedavg_results['train_loss_std'], 'b-', linewidth=2, label='FedAvg', marker='o', markersize=3)
plt.plot(rounds, srfedavg_results['train_loss_std'], 'r-', linewidth=2, label='SR-FedAvg', marker='s', markersize=3)
plt.xlabel('Global Rounds', fontsize=12)
plt.ylabel('Standard Deviation of Loss', fontsize=12)
plt.title('Stability: Training Loss Variance\nپایداری: واریانس خطا', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('stability_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ نمودار پایداری ذخیره شد: stability_analysis.png")

# محاسبه میانگین واریانس / Calculate average variance
avg_std_fedavg = np.mean(fedavg_results['test_acc_std'])
avg_std_srfedavg = np.mean(srfedavg_results['test_acc_std'])

print(f"\nمیانگین انحراف معیار FedAvg: {avg_std_fedavg:.4f}")
print(f"میانگین انحراف معیار SR-FedAvg: {avg_std_srfedavg:.4f}")
print(f"بهبود پایداری: {(1 - avg_std_srfedavg/avg_std_fedavg)*100:.2f}%")

## 10. نتیجه‌گیری / Conclusion

### مزایای SR-FedAvg / Advantages of SR-FedAvg:

1. **بهبود پایداری** / **Improved Stability**: 
   - کاهش واریانس در طول آموزش
   - Reduced variance during training

2. **کاهش حساسیت به مشارکت جزئی** / **Reduced Sensitivity to Partial Participation**:
   - عملکرد بهتر با join_ratio کمتر
   - Better performance with lower join_ratio

3. **همگرایی سریع‌تر** / **Faster Convergence**:
   - رسیدن به دقت بالاتر در دورهای کمتر
   - Achieving higher accuracy in fewer rounds

### کاربردها / Applications:
- سیستم‌های یادگیری فدرال با کلاینت‌های ناپایدار
- Federated learning systems with unstable clients
- محیط‌های با پهنای باند محدود
- Environments with limited bandwidth